In [1]:
import os

notebook_dir = os.getcwd()
os.chdir(notebook_dir)

import pandas as pd
import numpy as np
import trailblazer as tb
import GTExGenie as gtx
import pickle

print(notebook_dir)

/Users/patrick/Documents/CHBE194/Final_Project_Files


In [2]:
#read in human_proteins and gtex data
human_proteins = pickle.load(open(notebook_dir+'/inputs/human_proteins_LoD.p','rb'))

gtex = pd.read_csv(notebook_dir+'/inputs/GTEX_data_2020.gct',sep='\t',skiprows=2,index_col=0)

In [3]:
#use trailblazer to get a data frame of human genes from KEGG
human_genes = tb.getKeggGenes('hsa')

#use GTEx Genie to match the genes listed in gtex to KEGG ID
gtex_kegg_dictionary = gtx.KeggMatch(gtex, human_genes)

In [4]:
#read in attributes for gtex file
attributes = pd.read_excel(notebook_dir+'/inputs/Sample_attributes_v8.xlsx')

In [5]:
#get a list of unique tissue types
unique_tissues = gtx.tissueTypes(attributes)

In [6]:
#determine specificity of each transcript, where 1 is most specific and 54 is least specific
##This line of code will take several hours to complete, so I saved the output as a pickle

#specificity = gtx.transcriptSpecificity(gtex,attributes)
specificity = pickle.load(open(notebook_dir+'/inputs/specificity.pkl','rb'))

In [ ]:
#run the cytoscape function to get files to upload
#all files will be saved as .csv files in 'outputs' folder
tb.cytoscape(unique_tissues,
             gtex,
             attributes,
             KEGG_query='secreted',
             nModules=20000,
             addOn = {'Specificity':specificity})

In [7]:
#these genes are gathered from PCA analysis of patient transcriptomic data
predictive_obesity_genes = pickle.load(open(notebook_dir+'/inputs/top_1000.pickle','rb'))

In [9]:
#use the cytoscape function to generate a network with predictive obesity genes included in the attributes file
tb.cytoscape(['Adipose - Visceral'],
             gtex,
             attributes,
             nModules = 100000,
             version = '_obesity_genes',
             addOn = {'Specificity':specificity,
                      'Predictive_Genes':predictive_obesity_genes})

looping through unique tissues: 0/1 complete
writing cytoscape attributesK                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                              

#### Based on the network of visceral adipose tissue, several genes stand out for their combination of specificity, high expression, and nodal centrality. 

PLD1, LYN, CSF2RB, PTPRJ, PLD1, NLRP12, AMH, CHUK, CSCL1, GCK, VAV3, IL1B:

NLRP12 is an inflammatory signaling receptor

CHUK is an inhibitor of NFkB, which is a transcription factor involved in regulating many downstreatm targets, expecially ones involved in inflammation. 

LYN is a target of an insulin-mimicking drug called glimepiride. It also seems to be vital for proper liver regeneration. 

IL1B is certainly a propogator of inflammation, and is downstream of NFkB

PLD1 seems to be involved in exocytosis, and has been linked to Type II Diabetes.

I find it exciting that some of the predictive genes are ones that have been linked to T2D and insulin signaling. 